In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler


In [2]:
path = './Data/bt-impute.csv'

df = pd.read_csv(path)
print(len(df))
df.head()

44085


,SITE_ICD10_O2_3CHAR,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,...,SACT,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,NORMALISED_WEIGHT,HIGHEST_CYCLE,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,DAYS_TO_FIRST_REGIMEN
0,C71,9440,3,U,G4,69,1,L0401,R,4,...,False,0,False,False,0.000,0.00,0,0,751,0
1,C71,9440,3,U,G4,88,1,L1001,R,1,...,True,5,False,False,1.755,71.75,3,0,17,43
2,C71,9440,3,U,G4,88,1,L1001,R,1,...,True,5,False,False,1.755,71.75,3,0,17,43
3,C71,9401,3,U,G4,79,1,L1001,R,2,...,False,0,False,False,0.000,0.00,0,0,252,0
4,C71,9440,3,U,GX,79,2,L1001,R,2,...,False,0,False,False,0.000,0.00,0,18,425,0


In [3]:
df = df.drop(df[(df.SITE_ICD10_O2_3CHAR != 'C70') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C71') & 
                    (df.SITE_ICD10_O2_3CHAR != 'C72') & 
                    (df.SITE_ICD10_O2_3CHAR != 'D32') &
                    (df.SITE_ICD10_O2_3CHAR != 'D33') &
                    (df.SITE_ICD10_O2_3CHAR != 'D35')].index)
df.SITE_ICD10_O2_3CHAR.value_counts()

C71    23010
D32    11323
D33     4542
D35     4111
C72      727
C70      297
Name: SITE_ICD10_O2_3CHAR, dtype: int64

In [4]:
site_dummies = pd.get_dummies(df.SITE_ICD10_O2_3CHAR, prefix='SITE')
df = df.drop(columns = 'SITE_ICD10_O2_3CHAR')
df = pd.concat([df, site_dummies], axis=1)
df.head()


,MORPH_ICD10_O2,BEHAVIOUR_ICD10_O2,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,...,HIGHEST_CYCLE,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,DAYS_TO_FIRST_REGIMEN,SITE_C70,SITE_C71,SITE_C72,SITE_D32,SITE_D33,SITE_D35
0,9440,3,U,G4,69,1,L0401,R,4,2,...,0,0,751,0,0,1,0,0,0,0
1,9440,3,U,G4,88,1,L1001,R,1,2,...,3,0,17,43,0,1,0,0,0,0
2,9440,3,U,G4,88,1,L1001,R,1,2,...,3,0,17,43,0,1,0,0,0,0
3,9401,3,U,G4,79,1,L1001,R,2,2,...,0,0,252,0,0,1,0,0,0,0
4,9440,3,U,GX,79,2,L1001,R,2,2,...,0,18,425,0,0,1,0,0,0,0


In [5]:
df.MORPH_ICD10_O2.value_counts().to_csv('./Data/morph.csv', index=True)
# too little variations
df = df.drop(columns='MORPH_ICD10_O2')

In [6]:
df.BEHAVIOUR_ICD10_O2 = df.BEHAVIOUR_ICD10_O2.replace({2: 0, 5: 3, 6: 3, 9: 3})
df = df.drop(df[df.BEHAVIOUR_ICD10_O2 == 1].index)
df.BEHAVIOUR_ICD10_O2.value_counts()
df['BENIGN_BEHAVIOUR'] = (df.BEHAVIOUR_ICD10_O2 == 0).astype(int)
df = df.drop(columns= 'BEHAVIOUR_ICD10_O2')
df.head()

,STAGE_BEST,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,...,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,DAYS_TO_FIRST_REGIMEN,SITE_C70,SITE_C71,SITE_C72,SITE_D32,SITE_D33,SITE_D35,BENIGN_BEHAVIOUR
0,U,G4,69,1,L0401,R,4,2,W,A,...,0,751,0,0,1,0,0,0,0,0
1,U,G4,88,1,L1001,R,1,2,W,D,...,0,17,43,0,1,0,0,0,0,0
2,U,G4,88,1,L1001,R,1,2,W,D,...,0,17,43,0,1,0,0,0,0,0
3,U,G4,79,1,L1001,R,2,2,W,D,...,0,252,0,0,1,0,0,0,0,0
4,U,GX,79,2,L1001,R,2,2,U,D,...,18,425,0,0,1,0,0,0,0,0


In [7]:
df = df.drop(columns='STAGE_BEST')
df.head()

,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,...,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,DAYS_TO_FIRST_REGIMEN,SITE_C70,SITE_C71,SITE_C72,SITE_D32,SITE_D33,SITE_D35,BENIGN_BEHAVIOUR
0,G4,69,1,L0401,R,4,2,W,A,False,...,0,751,0,0,1,0,0,0,0,0
1,G4,88,1,L1001,R,1,2,W,D,True,...,0,17,43,0,1,0,0,0,0,0
2,G4,88,1,L1001,R,1,2,W,D,True,...,0,17,43,0,1,0,0,0,0,0
3,G4,79,1,L1001,R,2,2,W,D,False,...,0,252,0,0,1,0,0,0,0,0
4,GX,79,2,L1001,R,2,2,U,D,False,...,18,425,0,0,1,0,0,0,0,0


In [8]:
grade_mapper = {'GX' : -1, 'G0' : 0, 'G1' : 1, 'G2' : 2, 'G3' : 3, 'G4' : 4}
df.GRADE = df.GRADE.replace(grade_mapper)
df.head()

,GRADE,AGE,SEX,CREG_CODE,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,...,DAYS_TO_FIRST_SURGERY,DAYS_SINCE_DIAGNOSIS,DAYS_TO_FIRST_REGIMEN,SITE_C70,SITE_C71,SITE_C72,SITE_D32,SITE_D33,SITE_D35,BENIGN_BEHAVIOUR
0,4,69,1,L0401,R,4,2,W,A,False,...,0,751,0,0,1,0,0,0,0,0
1,4,88,1,L1001,R,1,2,W,D,True,...,0,17,43,0,1,0,0,0,0,0
2,4,88,1,L1001,R,1,2,W,D,True,...,0,17,43,0,1,0,0,0,0,0
3,4,79,1,L1001,R,2,2,W,D,False,...,0,252,0,0,1,0,0,0,0,0
4,-1,79,2,L1001,R,2,2,U,D,False,...,18,425,0,0,1,0,0,0,0,0


In [9]:
creg_dummies = pd.get_dummies(df.CREG_CODE, prefix='CREG')
df = df.drop(columns = 'CREG_CODE')
df = pd.concat([df, creg_dummies], axis=1)
df.head()

,GRADE,AGE,SEX,LATERALITY,QUINTILE_2015,TUMOUR_COUNT,ETHNICITY,NEWVITALSTATUS,SACT,REGIMEN_COUNT,...,SITE_D35,BENIGN_BEHAVIOUR,CREG_L0201,CREG_L0301,CREG_L0401,CREG_L0801,CREG_L0901,CREG_L1001,CREG_L1201,CREG_L1701
0,4,69,1,R,4,2,W,A,False,0,...,0,0,0,0,1,0,0,0,0,0
1,4,88,1,R,1,2,W,D,True,5,...,0,0,0,0,0,0,0,1,0,0
2,4,88,1,R,1,2,W,D,True,5,...,0,0,0,0,0,0,0,1,0,0
3,4,79,1,R,2,2,W,D,False,0,...,0,0,0,0,0,0,0,1,0,0
4,-1,79,2,R,2,2,U,D,False,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
lat_dummies = pd.get_dummies(df.LATERALITY, prefix='LAT')
df = df.drop(columns = 'LATERALITY')
df = pd.concat([df, lat_dummies], axis=1)

In [11]:
eth_dummies = pd.get_dummies(df.ETHNICITY, prefix='ETH')
df = df.drop(columns = 'ETHNICITY')
df = pd.concat([df, eth_dummies], axis=1)

In [12]:
# only 80
df = df.drop(df[df.NEWVITALSTATUS == 'X'].index)
vital_mapper = {'A' : 1, 'D' : 0}
df['ALIVE'] = df.NEWVITALSTATUS.replace(vital_mapper)
df = df.drop(columns='NEWVITALSTATUS')


In [13]:
df.SACT = df.SACT.astype(int)
df.CLINICAL_TRIAL_INDICATOR = df.CLINICAL_TRIAL_INDICATOR.astype(int)
df.CHEMO_RADIATION_INDICATOR = df.CHEMO_RADIATION_INDICATOR.astype(int)

In [14]:
df.head()

,GRADE,AGE,SEX,QUINTILE_2015,TUMOUR_COUNT,SACT,REGIMEN_COUNT,CLINICAL_TRIAL_INDICATOR,CHEMO_RADIATION_INDICATOR,NORMALISED_HEIGHT,...,LAT_M,LAT_R,ETH_A,ETH_B,ETH_C,ETH_M,ETH_O,ETH_U,ETH_W,ALIVE
0,4,69,1,4,2,0,0,0,0,0.000,...,0,1,0,0,0,0,0,0,1,1
1,4,88,1,1,2,1,5,0,0,1.755,...,0,1,0,0,0,0,0,0,1,0
2,4,88,1,1,2,1,5,0,0,1.755,...,0,1,0,0,0,0,0,0,1,0
3,4,79,1,2,2,0,0,0,0,0.000,...,0,1,0,0,0,0,0,0,1,0
4,-1,79,2,2,2,0,0,0,0,0.000,...,0,1,0,0,0,0,0,1,0,0


In [15]:
# spits out numpy array
def scale(df, method='minmax'):
    if method == 'minmax':
        scaler = MinMaxScaler()
    else:
        pass
    return scaler.fit_transform(df)

In [16]:
data = scale(df)
np.savetxt("./Data/data.csv", data, delimiter=",")

In [18]:
for f in df.columns:
    print(f)

GRADE
AGE
SEX
QUINTILE_2015
TUMOUR_COUNT
SACT
REGIMEN_COUNT
CLINICAL_TRIAL_INDICATOR
CHEMO_RADIATION_INDICATOR
NORMALISED_HEIGHT
NORMALISED_WEIGHT
HIGHEST_CYCLE
DAYS_TO_FIRST_SURGERY
DAYS_SINCE_DIAGNOSIS
DAYS_TO_FIRST_REGIMEN
SITE_C70
SITE_C71
SITE_C72
SITE_D32
SITE_D33
SITE_D35
BENIGN_BEHAVIOUR
CREG_L0201
CREG_L0301
CREG_L0401
CREG_L0801
CREG_L0901
CREG_L1001
CREG_L1201
CREG_L1701
LAT_9
LAT_B
LAT_L
LAT_M
LAT_R
ETH_A
ETH_B
ETH_C
ETH_M
ETH_O
ETH_U
ETH_W
ALIVE
